# **Autoencoder presentacion intermedia**

# Librerias necesarias


In [1]:
#Librerias visualizacion de datos
from os import path
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from google.colab import drive
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import pickle
from torchvision import transforms
import numpy as np
import os
import sys
import time
import random as rand

# Datos

In [4]:
# Están los datos en local o drive?
path_stamp_dataset_21_new = None
# path_stamp_dataset_21_new = 'E:Datos ZTF\stamp_dataset_21_new.pkl'

if path_stamp_dataset_21_new == None: # Sólo aplica si el path de drive coincide

    drive.mount('/content/drive') #Montaje de Google Drive
    path_stamp_dataset_21_new = '/content/drive/MyDrive/Proyecto Inteligencia/Datos ZTF/stamp_dataset_21_new.pkl'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Función para cargar archivos .pkl
def cargar_datos(ruta_archivo):
    with open(ruta_archivo, 'rb') as archivo:
        datos = pickle.load(archivo)
    return datos

#Cargar datasets
datos_stamp_dataset_21_new = cargar_datos(path_stamp_dataset_21_new)

# Mostrar datos


# Carga de datos

In [7]:
def print_keys(data, indent=0):
    for key, value in data.items():
        print(' ' * indent + str(key))
        if isinstance(value, dict):
            print_keys(value, indent + 2)
        elif isinstance(value, (list, tuple)):
            print(' ' * (indent + 2) + f'List/Tuple of length {len(value)}')
        elif hasattr(value, 'shape'):
            print(' ' * (indent + 2) + f'Shape: {value.shape}')
        else:
            print(' ' * (indent + 2) + str(type(value)))

print_keys(datos_stamp_dataset_21_new)

class StampDataset(Dataset):
    def __init__(self, data, split):
        self.images = data[split]['images']
        self.features = data[split]['features']
        self.label = data[split]['class']
        self.split = split

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        return self.images[idx], self.features[idx], self.label[idx]

train_dataset = StampDataset(datos_stamp_dataset_21_new, 'Train')
val_dataset = StampDataset(datos_stamp_dataset_21_new, 'Validation')

Train
  images
    Shape: (72710, 21, 21, 3)
  features
    Shape: (72710, 26)
  oid
    Shape: (50594,)
  class
    Shape: (72710,)
Test
  images
    Shape: (500, 21, 21, 3)
  features
    Shape: (500, 26)
  oid
    Shape: (500,)
  class
    Shape: (500,)
Validation
  images
    Shape: (500, 21, 21, 3)
  features
    Shape: (500, 26)
  oid
    Shape: (500,)
  class
    Shape: (500,)


# Autoencoder


In [8]:
class Reshape(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.shape = args

    def forward(self, x):
        return x.view(self.shape)


class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=1),  # 21x21 -> 18x18
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),  # 18x18 -> 18x18
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 18x18 -> 9x9
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # 9x9 -> 9x9
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),  # 9x9 -> 9x9
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 9x9 -> 4x4
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),  # 4x4 -> 4x4
            nn.ReLU(),
            nn.Flatten(),  # 4x4x64 -> 1024
            nn.Linear(1024, 64),  # 1024 -> 64
            nn.ReLU(),
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(64, 1024),  # 64 -> 1024
            Reshape(-1, 64, 4, 4),  # Reshape to 64 channels, 4x4
            nn.ConvTranspose2d(64, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  # 4x4 -> 9x9
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),  # 9x9 -> 18x18
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=1, padding=1),  # 18x18 -> 18x18
            nn.ReLU(),
            nn.Upsample(size=(21, 21), mode='bilinear', align_corners=True),  # Upsample from 18x18 -> 21x21
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=1, padding=1),  # 21x21x16 -> 21x21x3
            nn.Sigmoid()  # Final output between 0 and 1
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Visualizacion y confirmacion de inputs y outputs

In [9]:
dummy_input= torch.randn(1, 3, 21, 21)
model = ConvAutoencoder()
output = model(dummy_input)
#ocupar imagenes de 28x28 y cortar 1

In [10]:
def show_shapes(model, input_tensor):
    x = input_tensor
    print(f"Input shape: {x.shape}")

    # Show shapes in the encoder
    for layer in model.encoder:
        x = layer(x)
        print(f"After {layer.__class__.__name__}, shape: {x.shape}")

    # Show the encoder output shape
    print(f"Encoder output shape: {x.shape}")

    # Show shapes in the decoder
    for layer in model.decoder:
        x = layer(x)
        print(f"After {layer.__class__.__name__}, shape: {x.shape}")

    # Final output shape
    print(f"Final output shape: {x.shape}")

# Create the dummy input
dummy_input = torch.randn(1, 3, 21, 21)  # 21x21 input with 3 channels

# Instantiate the model
model = ConvAutoencoder()

# Show shapes for each layer
show_shapes(model, dummy_input)


Input shape: torch.Size([1, 3, 21, 21])
After Conv2d, shape: torch.Size([1, 32, 18, 18])
After ReLU, shape: torch.Size([1, 32, 18, 18])
After Conv2d, shape: torch.Size([1, 32, 18, 18])
After ReLU, shape: torch.Size([1, 32, 18, 18])
After MaxPool2d, shape: torch.Size([1, 32, 9, 9])
After Conv2d, shape: torch.Size([1, 64, 9, 9])
After ReLU, shape: torch.Size([1, 64, 9, 9])
After Conv2d, shape: torch.Size([1, 64, 9, 9])
After ReLU, shape: torch.Size([1, 64, 9, 9])
After MaxPool2d, shape: torch.Size([1, 64, 4, 4])
After Conv2d, shape: torch.Size([1, 64, 4, 4])
After ReLU, shape: torch.Size([1, 64, 4, 4])
After Flatten, shape: torch.Size([1, 1024])
After Linear, shape: torch.Size([1, 64])
After ReLU, shape: torch.Size([1, 64])
Encoder output shape: torch.Size([1, 64])
After Linear, shape: torch.Size([1, 1024])
After Reshape, shape: torch.Size([1, 64, 4, 4])
After ConvTranspose2d, shape: torch.Size([1, 64, 8, 8])
After ReLU, shape: torch.Size([1, 64, 8, 8])
After ConvTranspose2d, shape: torc

# Entrenamiento


In [11]:
def show_curves(curves):
    fig, ax = plt.subplots(1, 2, figsize=(13, 5))
    fig.set_facecolor('white')

    epochs = np.arange(len(curves["val_loss"])) + 1

    ax[0].plot(epochs, curves['val_loss'], label='validation')
    ax[0].plot(epochs, curves['train_loss'], label='training')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylabel('Loss')
    ax[0].set_title('Loss evolution during training')
    ax[0].legend()

    plt.show()

In [12]:
class EarlyStopping:
    def __init__(self, n_epochs_tolerance):
        self.n_epochs_tolerance = n_epochs_tolerance
        self.epochs_with_no_improvement = 0
        self.best_loss = np.inf

    def __call__(self, val_loss):
        # En cada llamada aumentamos el número de épocas en que no hemos mejorado
        self.epochs_with_no_improvement += 1

        if val_loss <= self.best_loss:
            # Si efectivamente mejoramos (menor loss de validación) reiniciamos el número de épocas sin mejora
            self.best_loss = val_loss
            self.epochs_with_no_improvement = 0

        # Retornamos True si debemos detenernos y False si aún no
        # Nos detenemos cuando el número de épocas sin mejora es mayor o igual que el número de épocas de tolerancia
        return self.epochs_with_no_improvement >= self.n_epochs_tolerance

In [13]:
def train_step(x_batch, model, optimizer, criterion, use_gpu):
    # Predicción
    x_batch = x_batch
    if use_gpu:
        x_batch = x_batch.cuda()

    # Paso hacia adelante en el autoencoder para obtener la reconstrucción
    x_reconstructed= model(x_batch)

    # Cálculo de la pérdida entre la imagen original y la reconstruida
    loss = criterion(x_reconstructed, x_batch)

    # Actualización de parámetros
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return x_reconstructed, loss


def evaluate(val_loader, model, criterion, use_gpu):
    cumulative_loss = 0
    cumulative_predictions = 0
    data_count = 0

    for x_val, _, _ in val_loader:
        if use_gpu:
            x_val = x_val.cuda()

        x_val = x_val.float().permute(0, 3, 1, 2)

        # Predicción de la reconstrucción
        x_reconstructed= model(x_val)

        # Cálculo de la pérdida
        loss = criterion(x_reconstructed, x_val)

        x_prediction = torch.argmax(x_reconstructed, axis=1).long()


        cumulative_loss += loss.item()
        data_count += x_val.shape[0]


    val_loss = cumulative_loss / len(val_loader)

    return val_loss


def train_model(
    model,
    train_dataset,
    val_dataset,
    epochs,
    criterion,
    batch_size,
    lr,
    n_evaluations_per_epoch=6,
    use_gpu=False,
    early_stopping_tolerance=5,
):
    if use_gpu:
        model.cuda()

    early_stopping = EarlyStopping(n_epochs_tolerance=early_stopping_tolerance)

    # Definición de dataloader
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=use_gpu)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False, pin_memory=use_gpu)

    # Optimizador
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Listas para guardar curvas de entrenamiento
    curves = {
        "train_loss": [],
        "val_loss": [],
    }

    t0 = time.perf_counter()

    iteration = 0

    n_batches = len(train_loader)

    for epoch in range(epochs):
        print(f"\rEpoch {epoch + 1}/{epochs}")
        cumulative_train_loss = 0
        train_loss_count = 0

        # Entrenamiento del modelo
        model.train()
        for i, (img, _, _) in enumerate(train_loader):
            if use_gpu:
                img = img.cuda()
            img = img.float().permute(0, 3, 1, 2)

            # Paso de entrenamiento
            x_reconstructed, loss = train_step(img, model, optimizer, criterion, use_gpu)

            cumulative_train_loss += loss.item()
            train_loss_count += 1

            # Evaluaciones periódicas
            if (i % (n_batches // n_evaluations_per_epoch) == 0) and (i > 0):
                train_loss = cumulative_train_loss / train_loss_count
                print(f"Iteration {iteration} - Batch {i}/{len(train_loader)} - Train loss: {train_loss}")

            iteration += 1

        # Evaluación del modelo en el conjunto de validación
        model.eval()
        with torch.no_grad():
            val_loss = evaluate(val_loader, model, criterion, use_gpu)

        print(f"Val loss: {val_loss}")

        # Guardar pérdidas para la curva de aprendizaje
        train_loss = cumulative_train_loss / train_loss_count
        curves["train_loss"].append(train_loss)
        curves["val_loss"].append(val_loss)

        if early_stopping(val_loss):
            print("Early stopping.")
            break

    print()
    print(f"Tiempo total de entrenamiento: {time.perf_counter() - t0:.4f} [s]")

    model.cpu()

    return curves


In [ ]:
lr = 1e-3
dropout_p = 0.5
batch_size = 64
criterion = nn.MSELoss()

epochs = 30
model = ConvAutoencoder()

curves = train_model(
    model,
    train_dataset,
    val_dataset,
    epochs,
    criterion,
    batch_size,
    lr,
    use_gpu=True,
)

show_curves(curves)